<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/kaggle_api_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the [Kaggle API](https://github.com/Kaggle/kaggle-api) in Colab

In [1]:
!pip install kaggle
# !pip install google
!conda install -y gdown

/bin/bash: conda: command not found


# Authenticating with Kaggle using kaggle.json

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [2]:
# from google.colab import files
import gdown
!gdown --id 1sD1x-nf2nXNNDFD3zdPKvOKM2wGSWcSP
# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1sD1x-nf2nXNNDFD3zdPKvOKM2wGSWcSP
To: /content/kaggle.json
100% 69.0/69.0 [00:00<00:00, 57.5kB/s]


# Using the Kaggle API

For a more complete list of what you can do with the API, visit https://github.com/Kaggle/kaggle-api.

## Listing competitions

In [3]:
# !kaggle competitions list

## Downloading a dataset

In [4]:
# kaggle competitions download -c covid-19-risk-2022

In [5]:
!kaggle competitions download -c covid-19-risk-2022

train_small.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
example_submission.ipynb: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
N!unzip train.csv.zip

!unzip test.csv.zip

!unzip train_small.csv.zip

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  train_small.csv.zip
replace train_small.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
import pandas as pd 

train_small = pd.read_csv('/content/train_small.csv')
# train = pd.read_csv('/content/train.csv')
# test = pd.read_csv('/content/test.csv')
train_small.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,labconfirmed_yn,symptomatic_yn,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2021-09,NY,36.0,BRONX,36005.0,0 - 17 years,Male,NaN,NaN,0.0,NaN,Missing,NaN,1,0,NaN,NaN,NaN,NaN
1,2021-09,CA,6.0,SAN JOAQUIN,6077.0,18 to 49 years,Male,NaN,NaN,NaN,NaN,Missing,NaN,1,NaN,NaN,NaN,NaN,NaN
2,2021-09,MA,25.0,MIDDLESEX,25017.0,0 - 17 years,Female,Missing,Unknown,NaN,NaN,Missing,NaN,1,NaN,NaN,NaN,NaN,NaN
3,2021-09,PA,42.0,ERIE,42049.0,65+ years,Male,White,Non-Hispanic/Latino,0.0,0.0,Missing,1.0,1,1,0.0,0,0.0,NaN
4,2021-09,CA,6.0,KERN,6029.0,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,NaN,1,NaN,NaN,NaN,NaN,NaN


So we've got a case_month, res_state, state_fips_code, res_county, county_fips_code, age_group, sex, race, ethnicity,lots more stuff.

In [8]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
train.columns

Index(['case_month', 'res_state', 'state_fips_code', 'res_county',
       'county_fips_code', 'age_group', 'sex', 'race', 'ethnicity',
       'case_positive_specimen_interval', 'case_onset_interval', 'process',
       'exposure_yn', 'labconfirmed_yn', 'symptomatic_yn', 'hosp_yn', 'icu_yn',
       'death_yn', 'underlying_conditions_yn'],
      dtype='object')

In [49]:
import numpy as np

sample_train = train.sample(1)
print(train.columns)
print(train['process'].unique())
for col in train.columns:
  unique_vec = train[col].unique()
  print(unique_vec)
  unique_vec = filter(lambda v: v==v, unique_vec)
  print(unique_vec)
  for value in unique_vec:
    print([col,value])
    sample_train=sample_train.append(train[(train[col] == value)].sample(n=1))

sample_train.head()

Index(['case_month', 'res_state', 'state_fips_code', 'res_county',
       'county_fips_code', 'age_group', 'sex', 'race', 'ethnicity',
       'case_positive_specimen_interval', 'case_onset_interval', 'process',
       'exposure_yn', 'labconfirmed_yn', 'symptomatic_yn', 'hosp_yn', 'icu_yn',
       'death_yn', 'underlying_conditions_yn'],
      dtype='object')
['Missing' 'Clinical evaluation' 'Routine surveillance'
 'Other detection method (specify)' 'Multiple' 'Other'
 'Contact tracing of case patient' 'Unknown' 'Laboratory reported'
 'Provider reported' 'Routine physical examination' 'Autopsy']
['2020-01' '2020-02' '2020-03' '2020-04' '2020-05' '2020-06' '2020-07'
 '2020-08' '2020-09' '2020-10' '2020-11' '2020-12' '2021-01' '2021-02'
 '2021-03' '2021-04' '2021-05' '2021-06' '2021-07' '2021-08' '2021-09']
['case_month', '2020-01']
['case_month', '2020-02']
['case_month', '2020-03']
['case_month', '2020-04']
['case_month', '2020-05']
['case_month', '2020-06']
['case_month', '2020-07']
['

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,labconfirmed_yn,symptomatic_yn,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
3158557,2020-07,NV,32.0,CLARK,32003.0,18 to 49 years,Male,Unknown,Unknown,0.0,0.0,Routine surveillance,NaN,1,1,NaN,NaN,NaN,NaN


In [50]:
sample_train

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,labconfirmed_yn,symptomatic_yn,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
3158557,2020-07,NV,32.0,CLARK,32003.0,18 to 49 years,Male,Unknown,Unknown,0.0,0.0,Routine surveillance,NaN,1,1,NaN,NaN,NaN,NaN


In [18]:
train['age_group'].unique()

array([nan, '65+ years', '18 to 49 years', '50 to 64 years',
       '0 - 17 years', 'Missing'], dtype=object)

In [14]:
# # There is the python way
# import pandas
# import random



# n = print(train.shape) #number of records in file
# s = 1000000 #desired sample size
# skip = sorted(random.sample(xrange(n),n-s))
# df = pandas.read_csv(filename, skiprows=skip)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyError: ignored

In [ ]:
train.describe()

In [ ]:
def randomiseMissingData(df):
    import random
    "randomise missing data for DataFrame (within a column)"
    # df = df2.copy()
    for col in df.columns:
        data = df[col]
        mask = data.isnull()
        samples = random.choices( data[~mask].values , k = mask.sum() )
        df.loc[mask,col] = samples
    return df

In [ ]:
y = train['death_yn']

In [ ]:
fill_train = randomiseMissingData(train)

In [ ]:
fill_train_X = fill_train.drop(['death_yn'], inplace=False, axis=1)
fill_train_X = pd.get_dummies(fill_train_X)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8,
                        random_state=1)

bag.fit(X, y)

In [ ]:
fill_test = randomiseMissingData(test)
# test_X = fill_test.drop(['death_yn'], inplace=False, axis=1)
test_X = pd.get_dummies(fill_test)


In [ ]:
bag.predict(test_X)

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)

In [ ]:
y

In [ ]:

df["column"].fillna(lambda x: random.choice(df[df[column] != np.nan]["column"]), inplace =True)


In [ ]:
train_small["column"].fillna(lambda x: random.choice(df[df[column] != np.nan]["column"]), inplace =True)


In [ ]:
print(train_na)

In [ ]:
test_na = test.dropna(axis = 0, how = 'any', inplace = True)

In [ ]:
print(test_na)

So you can immediately notice some pretty big differences between the two dataset. Firstly, more counties and states in training than test. Probably a true random sample (which could mean that some rare counties or combinations of groups don't exist in the validation set). Same with sex, pretty big difference between the two. However thats a little weirder because there so many. The two ratios should be much closer than that.

In [ ]:
# Example code for training model and creating submission file.
# Author: Peter Sadowski Jan 22 2022
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Load training data.
df_train = pd.read_csv('./train_small.csv.zip') # Can read from zip files directly.
df_train = df_train.replace({'death_yn':{np.nan:0}}) # Assume no info means survived.
y = df_train['death_yn']

# Load test data.
df_test = pd.read_csv('./test.csv.zip')

# Encode state variable as one-hot. 
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc.fit(df_train[['res_state']])
state_train = enc.transform(df_train[['res_state']])
state_test = enc.transform(df_test[['res_state']])

# Combine this with whether patient went to ICU.
X = np.concatenate([df_train[['icu_yn']]==1, state_train], axis=1)
X_test = np.concatenate([df_test[['icu_yn']] == 1, state_test], axis=1)

# Make predictions based on whether patient went to ICU, and their state.
model = LogisticRegression()
model.fit(X,y)
ypred = model.predict_proba(X_test)[:,1]
print(f'Model coefficients: {model.coef_}')

# Create submission file.
submission = pd.DataFrame(ypred, columns=['prediction']) # Create new dataframe.
submission['Id'] = submission.index  # Kaggle expects two columns: Id, prediction.
submission.to_csv('sample_submission.csv', index=False)

import matplotlib.pylab as plt
plt.hist(ypred, bins=100);

In [ ]:
submission
!kaggle competitions submit covid-19-risk-2022 -f sample_submission.csv -m 'Heres Johnny'


## Uploading a Colab notebook to Kaggle Kernels

Bear with us, as this is a little round-about...

### Downloading a notebook from Colab

To download from Colab, use **File** | **Download .ipynb**

In [ ]:
# user = "ics435"
# repo = "ps1-numpy-seismosmsr"
# src_dir = "master"
# pyfile = "kaggle_api_in_colab.ipynb"
# raw_git = 'https://raw.githubusercontent.com/seismosmsr/machine_learning/main/kaggle_api_in_colab.ipynb'

# url = f"{raw_git}"

# !wget --no-cache --backups=1 {url} -o submission.ipynb

### Then upload the notebook to your Colab runtime

In [ ]:
# # uploaded = files.upload()
# notebook_path = '/content/kaggle_api_in_colab.ipynb'

In [ ]:
# uploaded = files.upload()
# notebook_path = list(uploaded.keys())[0]

In [ ]:
# !mkdir -p export
# !mv $notebook_path export/
# !kaggle kernels init -p export

In [ ]:
# import re
# import random
# your_kaggle_username = 'Aron Boettcher'
# notebook_title = 'Test Kernel ' + str(random.randint(1,100))
# new_kernel_slug = re.sub(r'[^a-z0-9]+', '-', notebook_title.lower())
# notebook_path = 'kaggle_api_in_colab.ipynb'

In [ ]:
# # Documented here: https://github.com/Kaggle/kaggle-api/wiki/Kernel-Metadata
# metadata = '''
# {
#   "id": "%s/%s",
#   "title": "%s",
#   "code_file": "%s",
#   "language": "python",
#   "kernel_type": "notebook",
#   "is_private": "true",
#   "enable_gpu": "false",
#   "enable_internet": "true",
#   "dataset_sources": [],
#   "competition_sources": [],
#   "kernel_sources": []
# }
# ''' % (your_kaggle_username, new_kernel_slug, notebook_title, notebook_path)

In [ ]:
# !echo '$metadata' > export/kernel-metadata.json
# !cat export/kernel-metadata.json

In [ ]:
# !kaggle kernels push -p export

In [11]:
# the functions:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):
    '''
    It samples data from a pandas dataframe using strata. These functions use
    proportionate stratification:
    n1 = (N1/N) * n
    where:
        - n1 is the sample size of stratum 1
        - N1 is the population size of stratum 1
        - N is the total population size
        - n is the sampling size
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    :seed: sampling seed
    :keep_index: if True, it keeps a column with the original population index indicator
    
    Returns
    -------
    A sampled pandas dataframe based in a set of strata.
    Examples
    --------
    >> df.head()
    	id  sex age city 
    0	123 M   20  XYZ
    1	456 M   25  XYZ
    2	789 M   21  YZX
    3	987 F   40  ZXY
    4	654 M   45  ZXY
    ...
    # This returns a sample stratified by sex and city containing 30% of the size of
    # the original data
    >> stratified = stratified_sample(df=df, strata=['sex', 'city'], size=0.3)
    Requirements
    ------------
    - pandas
    - numpy
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df



def stratified_sample_report(df, strata, size=None):
    '''
    Generates a dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Returns
    -------
    A dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)
    return tmp_grpd


def __smpl_size(population, size):
    '''
    A function to compute the sample size. If not informed, a sampling 
    size will be calculated using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Parameters
    ----------
        :population: population size
        :size: sample size (default = None)
    Returns
    -------
    Calculated sample size to be used in the functions:
        - stratified_sample
        - stratified_sample_report
    '''
    if size is None:
        cochran_n = round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n
